# GraphRAG do conteúdo dos pdfs das bulas de medicamentos salvos no Neo4j

In [1]:
from src.connetion.embeddings import EmbeddingsModel
from src.connetion.chat_model import LLMModel
from src.connetion.graph_db import KgDatabaseConnetion
from src.kg.graph_rag import GraphRAG

2025-06-15 16:48:04.939 | INFO     | src.config:<module>:12 - PROJ_ROOT path is: /home/marilia/git/pos/tcc_ufg_akcit_nlp


In [2]:
provider = "openai"
temperature = 0
max_tokens = None

llm = LLMModel(
    provider=provider,
    temperature=temperature,
    max_tokens=max_tokens
).llm

In [4]:
provider = "hf"
embedding = EmbeddingsModel(
    provider=provider,
).embeddings

In [5]:
db = KgDatabaseConnetion(
    llm=llm,
    embedding=embedding,
)

/home/marilia/git/pos/tcc_ufg_akcit_nlp/src/connetion/graph_db.py:103: LangChainDeprecationWarning: The class `Neo4jVector` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jVector``.
  self.vector = Neo4jVector(
/home/marilia/git/pos/tcc_ufg_akcit_nlp/src/connetion/graph_db.py:30: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  self.graph = Neo4jGraph(
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecat

In [6]:
db.vector.show_vector_constraints_and_indexes()

([{'id': 3,
   'name': 'constraint_907a464e',
   'type': 'UNIQUENESS',
   'entityType': 'NODE',
   'labelsOrTypes': ['__Entity__'],
   'properties': ['id'],
   'ownedIndex': 'constraint_907a464e',
   'propertyType': None}],
 [{'name': 'vector',
   'type': 'VECTOR',
   'labelsOrTypes': ['Document'],
   'properties': ['embedding'],
   'options': {'indexProvider': 'vector-2.0',
    'indexConfig': {'vector.hnsw.m': 16,
     'vector.hnsw.ef_construction': 100,
     'vector.dimensions': 768,
     'vector.similarity_function': 'COSINE',
     'vector.quantization.enabled': True}}}])

In [13]:
print(db.graph.graph.schema)

Node properties:
- **Medicamento**
  - `id`: STRING Available options: ['Medicamento']
- **Dose**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Sintoma**
  - `id`: STRING Available options: ['Sintomas']
- **Problema de saúde**
  - `id`: STRING Available options: ['Problemas_De_Saude']
- **Pessoa**
  - `id`: STRING Available options: ['Medico']
- **Document**
  - `id`: STRING Available options: ['a86bce566e661a48338e0e5dd2436b91']
  - `text`: STRING Available options: ['4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO']
- **Conceito**
  - `id`: STRING Available options: ['Doenca_Grave', 'Sintoma', 'Saude', 'Dose_Recomendada', 'Superdose']
- **Profissional**
  - `id`: STRING Available options: ['Medico']
- **Medicament**
  - `id`: STRING Available options: ['Medicamento']
- **Dosage**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Healthissue**
  - `id`: STRING Available options: ['Problemas_De_Saude', 'Doencas_Graves']
- **Person**
  - `id`: STRING Availab

In [14]:
print(db.graph.graph.get_schema)

Node properties:
- **Medicamento**
  - `id`: STRING Available options: ['Medicamento']
- **Dose**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Sintoma**
  - `id`: STRING Available options: ['Sintomas']
- **Problema de saúde**
  - `id`: STRING Available options: ['Problemas_De_Saude']
- **Pessoa**
  - `id`: STRING Available options: ['Medico']
- **Document**
  - `id`: STRING Available options: ['a86bce566e661a48338e0e5dd2436b91']
  - `text`: STRING Available options: ['4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO']
- **Conceito**
  - `id`: STRING Available options: ['Doenca_Grave', 'Sintoma', 'Saude', 'Dose_Recomendada', 'Superdose']
- **Profissional**
  - `id`: STRING Available options: ['Medico']
- **Medicament**
  - `id`: STRING Available options: ['Medicamento']
- **Dosage**
  - `id`: STRING Available options: ['Dose_Recomendada']
- **Healthissue**
  - `id`: STRING Available options: ['Problemas_De_Saude', 'Doencas_Graves']
- **Person**
  - `id`: STRING Availab

In [15]:
grag = GraphRAG(llm=llm, db=db)

In [17]:
grag.retriever("O que devo saber antes de tomar o medicamento?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (d:Document)-[:MENTIONS]->(m:Medicamento)
WHERE d.text CONTAINS 'O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO'
RETURN d, m

Full Context:
[{'d': {'id': 'a86bce566e661a48338e0e5dd2436b91', 'text': '4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada (superdose) para provocar maior alívio, pois pode causar sérios problemas de saúde. Você deve consultar seu médico se a dor ou febre continuar ou piorar, ou se surgirem novos sintomas, pois estes sintomas podem ser sinais de doenças graves.', 'embedding': [0.05720214918255806, -0.03908204659819603, -0.006184726487845182, -0.0030857762321829796, 0.022133415564894676, 0.011207369156181812, -0.008866087533533573, 0.028748128563165665, 0.028442904353141785, -0.0059473454020917416, -0.0016515799798071384, 0.0034806428011506796, 0.02001987397670746, 0.028822530061006546, -0.03205113112926483, -0.04275377839803696, -0

RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-4o-mini in organization org-WOfWbo10AvN503F5ienlShmL on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}

In [ ]:
grag.retriever(question="What was the cast of the Casino?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "Casino"}) RETURN p.name

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.
** QA chain results **
{'query': 'What was the cast of the Casino?', 'result': 'The cast of Casino includes Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.', 'intermediate_steps': [{'query': 'cypher\nMATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "Casino"}) RETURN p.name\n'}, {'context': [{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]}]}
** Vector similarity results **
[Document(metadata={}, page_content='4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada (superdose) para provocar maior alívio, pois pode causar sérios problemas de saúde. Você deve consultar seu médico se a dor o

KeyboardInterrupt: 

Generated Cypher:
cypher
MATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "Casino"}) RETURN p.name

Full Context:
[{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]

> Finished chain.
** QA chain results **
{'query': 'What was the cast of the Casino?', 'result': 'The cast of Casino includes Robert De Niro, Joe Pesci, Sharon Stone, and James Woods.', 'intermediate_steps': [{'query': 'cypher\nMATCH (p:Person)-[:ACTED_IN]->(m:Movie {title: "Casino"}) RETURN p.name\n'}, {'context': [{'p.name': 'Robert De Niro'}, {'p.name': 'Joe Pesci'}, {'p.name': 'Sharon Stone'}, {'p.name': 'James Woods'}]}]}
** Vector similarity results **
[Document(metadata={}, page_content='4. O QUE DEVO SABER ANTES DE USAR ESTE MEDICAMENTO?  \nVocê não deve tomar mais do que a dose recomendada (superdose) para provocar maior alívio, pois pode causar sérios problemas de saúde. Você deve consultar seu médico se a dor ou febre continuar ou piorar, ou se surgirem n